<a href="https://colab.research.google.com/github/mp321/Web-Scrape-examples/blob/main/posh_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
#shouldn't need to update - persists


# Step 1: Install necessary packages
# Selenium WebDriver is set up using chromium-browser in a headless mode, which allows you to run it in the background without a GUI.
!pip install selenium
!pip install webdriver-manager

# Step 2: Install Chromium and dependencies
!apt-get update # Update the system
!apt install -y chromium-browser  # Install Chromium

# if need to troubleshoot consider these
# !apt-get install --reinstall chromium-browser
# !which chromium-browser  # Check if it's installed at /usr/bin/chromium-browser
# !apt-get install -y libnss3 libgdk-pixbuf2.0-0 libxss1 libasound2
# # !pip install gspread oauth2client

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
0 upgraded, 0 newly installed, 1 reinstalled, 0 to remove and 33 not upgraded.
Need to get 49.2 kB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 chromium-browser amd64 1:85.0.4183.83-0ubuntu2.22.04.1 [49.2 kB]
Fetched 49.2 kB in 0s (538 kB/s)
Preconfiguring packages ...
(Reading database ... 125403 files and directories currently installed.)
Preparing to unpack .../chromium-browser_1%3a85.0.4183.83-0ubuntu2.22.04.1_amd64.deb ...
=> Installing the chromium snap
==> Checking connectivity with the snap store
===> System doesn't have a working snapd, skipping
=> Connecting the password-manager-service interface (LP: #1836616)
error: cannot communicate with server: Post "http://localhost/v2/interfaces": dial unix /run/snapd.socket: connect: no such file or directory
Unpacking chromium-browser (1:8

In [32]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from datetime import datetime
from google.auth import default
from google.auth.transport.requests import Request
import gspread

# Step 1: Authenticate Google Colab using google-auth
creds, _ = default()

# Refresh the credentials if expired
if creds and creds.expired and creds.refresh_token:
    creds.refresh(Request())

# Step 2: Authorize gspread with the credentials
gc = gspread.authorize(creds)

# Step 3: Open Google Sheets by name
spreadsheet = gc.open('poshmark_data')  # Replace with your sheet name
worksheet = spreadsheet.worksheet('MENS_JacketsCoats100_500_JustInSold')
# worksheet = spreadsheet.sheet1  # Choose the first sheet -- originall worked del

# Step 4: Setup Chrome WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in background
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Set the binary location to where Chromium is installed in Colab
options.binary_location = '/usr/bin/chromium-browser'

# Initialize the WebDriver using SeleniumManager (this auto handles the driver download)
driver = webdriver.Chrome(options=options)

# Step 5: Target URL
url = "https://poshmark.com/category/Men-Jackets_&_Coats?sort_by=added_desc&price%5B%5D=100-250&price%5B%5D=250-500&availability=sold_out"
driver.get(url)

# Step 6: Allow Time for JavaScript to Load
time.sleep(5)

# # Step 7: Extract Brand Names
# brand_elements = driver.find_elements(By.CLASS_NAME, "tile__title")
# brands = [brand.text for brand in brand_elements]

# # Step 8: Ensure Data is Collected Correctly
# if brands:
#     # Get today's date
#     today_date = datetime.today().strftime('%m-%d-%Y')
#     data = {'Brand': brands, 'Date': [today_date] * len(brands)}

#     # Create DataFrame
#     df = pd.DataFrame(data)

#     # Step 9: Convert DataFrame to list and append to Google Sheets
#     data_to_append = df.values.tolist()
#     worksheet.append_rows(data_to_append)  # Append data to Google Sheets

#     # Print DataFrame
#     print(df)
# else:
#     print("⚠️ No brands found.")

# # Step 10: Quit the WebDriver
# driver.quit()



#########################

# Step 7: Extract Brand Names and Prices
try:
    # Extract brands
    brand_elements = driver.find_elements(By.CLASS_NAME, "tile__title")
    brands = [brand.text for brand in brand_elements]

    # Extract prices (if available)
    price_elements = driver.find_elements(By.CLASS_NAME, "price")
    prices = [price.text for price in price_elements]

    # Fill missing prices with 'N/A' in case some listings don't have prices
    if len(prices) < len(brands):
        prices.extend(['N/A'] * (len(brands) - len(prices)))

    # Step 8: Ensure Data is Collected Correctly
    if brands:
        # Get today's date
        today_date = datetime.today().strftime('%m-%d-%Y')

        # Create DataFrame with Brand and Price
        data = {'Brand': brands, 'Price': prices, 'Date': [today_date] * len(brands)}

        # Create DataFrame
        df = pd.DataFrame(data)

        # Step 9: Convert DataFrame to list and append to Google Sheets
        data_to_append = df.values.tolist()
        worksheet.append_rows(data_to_append)  # Append data to Google Sheets

        # Print DataFrame for verification
        print(df)
    else:
        print("⚠️ No brands found.")
except Exception as e:
    print(f"⚠️ An error occurred: {e}")

# Step 10: Quit the WebDriver
driver.quit()



                                                Brand Price        Date
0   Harley Davidson Piledriver 2.0 Snapback Textil...   N/A  02-23-2025
1   BULWARK FR Navy Blue Jacket Cat 3 Coat Size XL...   N/A  02-23-2025
2   Men's Lululemon Twill Full-Zip Jacket CMPK Com...   N/A  02-23-2025
3   BARBOUR LUTZ Men’s Waxed Jacket Olive Rugged S...   N/A  02-23-2025
4                                 UO Printed Cat Vest   N/A  02-23-2025
5   Essentials Fear of God Classic Logo Black On B...   N/A  02-23-2025
6                          Moncler maya puffer jacket   N/A  02-23-2025
7   Nike Air Jordan Puffer Jacket Red Black Full Z...   N/A  02-23-2025
8   Canada Goose Chilliwack 01 Men's Short Flight ...   N/A  02-23-2025
9   POLO Ralph Lauren 4 pocket field jacket, green...   N/A  02-23-2025
10  Patagonia Men's Nano Puff Jacket Color: Burnis...   N/A  02-23-2025
11  Chrome Hearts Black Multicolor Cross Zip up ho...   N/A  02-23-2025
12           Mitchell & Ness: LA Dodgers Satin Jacket   N/A  02-

In [31]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from datetime import datetime
from google.auth import default
from google.auth.transport.requests import Request
import gspread

# Step 1: Authenticate Google Colab using google-auth
creds, _ = default()

# Refresh the credentials if expired
if creds and creds.expired and creds.refresh_token:
    creds.refresh(Request())

# Step 2: Authorize gspread with the credentials
gc = gspread.authorize(creds)

# Step 3: Open Google Sheets by name
spreadsheet = gc.open('poshmark_data')  # Replace with your sheet name
worksheet = spreadsheet.worksheet('MENS_JacketsCoats100_500_JustInSold')
# worksheet = spreadsheet.sheet1  # Choose the first sheet -- originall worked del

# Step 4: Setup Chrome WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in background
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Set the binary location to where Chromium is installed in Colab
options.binary_location = '/usr/bin/chromium-browser'

# Initialize the WebDriver using SeleniumManager (this auto handles the driver download)
driver = webdriver.Chrome(options=options)

# Step 5: Target URL
url = "https://poshmark.com/category/Men-Jackets_&_Coats?sort_by=added_desc&price%5B%5D=100-250&price%5B%5D=250-500&availability=sold_out"
driver.get(url)

# Step 6: Allow Time for JavaScript to Load
time.sleep(5)


# Step 7: Extract Brand Names and Prices
try:
    # Extract brands
    brand_elements = driver.find_elements(By.CLASS_NAME, "tile__title")
    brands = [brand.text for brand in brand_elements]

    # Extract prices using XPath or CSS selector
    price_elements = driver.find_elements(By.XPATH, '//div[contains(@class, "tile__price")]/span')
    prices = [price.text for price in price_elements]

    # If price data is missing for any items, fill with 'N/A'
    if len(prices) < len(brands):
        prices.extend(['N/A'] * (len(brands) - len(prices)))

    # Step 8: Ensure Data is Collected Correctly
    if brands:
        # Get today's date
        today_date = datetime.today().strftime('%m-%d-%Y')

        # Create DataFrame with Brand and Price
        data = {'Brand': brands, 'Price': prices, 'Date': [today_date] * len(brands)}

        # Create DataFrame
        df = pd.DataFrame(data)

        # Step 9: Convert DataFrame to list and append to Google Sheets
        data_to_append = df.values.tolist()
        worksheet.append_rows(data_to_append)  # Append data to Google Sheets

        # Print DataFrame for verification
        print(df)
    else:
        print("⚠️ No brands found.")
except Exception as e:
    print(f"⚠️ An error occurred: {e}")

# Step 10: Quit the WebDriver
driver.quit()


                                                Brand Price        Date
0   Harley Davidson Piledriver 2.0 Snapback Textil...   N/A  02-23-2025
1   BULWARK FR Navy Blue Jacket Cat 3 Coat Size XL...   N/A  02-23-2025
2   Men's Lululemon Twill Full-Zip Jacket CMPK Com...   N/A  02-23-2025
3   BARBOUR LUTZ Men’s Waxed Jacket Olive Rugged S...   N/A  02-23-2025
4                                 UO Printed Cat Vest   N/A  02-23-2025
5   Essentials Fear of God Classic Logo Black On B...   N/A  02-23-2025
6                          Moncler maya puffer jacket   N/A  02-23-2025
7   Nike Air Jordan Puffer Jacket Red Black Full Z...   N/A  02-23-2025
8   Canada Goose Chilliwack 01 Men's Short Flight ...   N/A  02-23-2025
9   POLO Ralph Lauren 4 pocket field jacket, green...   N/A  02-23-2025
10  Patagonia Men's Nano Puff Jacket Color: Burnis...   N/A  02-23-2025
11  Chrome Hearts Black Multicolor Cross Zip up ho...   N/A  02-23-2025
12           Mitchell & Ness: LA Dodgers Satin Jacket   N/A  02-